In [2]:
import sagemaker
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [6]:
from sagemaker.amazon.amazon_estimator import get_image_uri

In [7]:
from sagemaker.session import s3_input, Session

In [8]:
bucket_name = 'projectexamplebucket20240721'

In [9]:
my_region = boto3.session.Session().region_name
my_region

'us-east-2'

In [19]:
import boto3

s3_client = boto3.client('s3', region_name='us-east-2')

bucket_name = 'projectexamplebucket20240721'

try:
    response = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': 'us-east-2'
        }
    )
    print("Bucket created successfully")
except Exception as e:
    print("Error: ", e)

Bucket created successfully


In [20]:
# s3 = boto3.resource('s3')

# try:
#     if my_region == 'us-east-2':
#         s3.create_bucket(Bucket=bucket_name)
#     print('S3 bucket created successfully')
# except Exception as e:
#     print('S3 error: ', e)


In [22]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = f's3://{bucket_name}/{prefix}/output'
output_path

's3://projectexamplebucket20240721/xgboost-as-a-built-in-algo/output'

## Downloading the Dataset and Storing in S3

In [25]:
import pandas as pd
import urllib

urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
model_data = pd.read_csv('./bank_clean.csv',index_col=0)

In [27]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [30]:
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], axis=1).to_csv('train.csv', 
                                                                         index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')


In [31]:
import os
pd.concat([test_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], axis=1).to_csv('test.csv', 
                                                                         index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file(
    'test.csv')

In [33]:
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

In [38]:
?sagemaker.image_uris.retrieve

In [41]:
container = sagemaker.image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.0-1')

In [43]:
hyperparameters = {
    "max_depth": "3", 
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",    
}

In [44]:
?sagemaker.estimator.Estimator

In [45]:
eatimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1,
                                          instance_type='ml.m4.xlarge',
                                          volume_size=4,
                                          output_path=output_path,
                                          use_spot_instances=True,
                                          max_run=300,
                                          max_wait=600
                                         )

In [ ]:
eatimator.fit({'train': s3_input_train, 'validation': s3_input_test})

In [ ]:
xgb_predictor = eatimator.deploy(initial_instance_count=1,instance_type='ml.t2.medium')

In [ ]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).as_matrix() #load the data into an array
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()